# Whatsapp

## ETL

### Importar

In [1]:
# %load basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',200)
pd.options.display.float_format = '{:.2f}'.format
file = '/home/ef/Documents/Diplomado/data/'

In [2]:
import time
start = time.time()

In [3]:
file += 'WhatsApp Chat with Naps 🐻🐼🐯.txt'

In [4]:
with open(file) as chat:
    chat = chat.read()

Explicación por renglón:

1. Se importa el módulo de regular expressions

2. Se guardará el patrón de texto en un objeto, el inicio se indica que es un raw string, las 3 comillas simples indican que será multilínea

3. Para cada variable, (?P\<grupo\>patrón). Entonces la fecha:
    
    - [^,] separa el texto a partir del símbolo ","
    
    - \+ significa que el caracter (o grupo) puede repetirse
    
    - (...) agrupa lo que esté dentro del paréntesis
    
    - Después de la fecha, hay una coma y por eso se indica después del grupo
    
    - \s es un espacio en blanco (el + indica que pueda haber más de un espacio)
    
3. Separa lo que esté antes de "-" y lo agrupa en "hora"
    
4. Misma lógica para "autor"
    
5. Ahora, para el mensaje:
    
    - \S es cualquier caracter excepto espacios en blanco, lo contrario a \s, entonces
    [\s\S]+ es TODO
    
    - ? da la pauta para no agrupar el resto del texto, véase punto siguiente
    
6. Finalmente, esta línea logra que el patrón se repita a lo largo del documento:
    - ?= es un match condicional, si lo que sigue cumple con el patrón. Por ejemplo,
    'Isaac (?=Asimov)' va a leer 'Isaac ' sólo si está seguido de 'Asimov'
    
    - para definir el formato de fecha según el doc: \d es un dígito
    
    - {1,2} denota que el caracter puede repetirse de una a dos veces
    
    - \/ es el símbolo literal con el que se separa día/mes/año
    
    - | se ocupa como "or" para que con...
    
    - \Z se pueda tomar en cuenta el último grupo en el documento
    
.

¿Por qué no se podía leer como un txt normal? Principalmente, por los mensajes multilínea: el método pd.read_csv() no puede saber dónde termina el mensaje, además es más complicado separar las columnas dado que existen múltiples delimitadores.

En cambio, con regex podemos darle las instrucciones específicas para seguir el patrón correcto.

In [5]:
## Se genera el patrón de regular expressions
import re
pattern =  r'''
            (?P<Fecha>[^-]+)\s+-\s+
            (?P<Autor>[^:]+):\s+
            (?P<Mensaje>[\s\S]+?)
            (?=(\d{1,2}/\d{1,2}/\d{2})|\Z)
            '''     

## Encuentra todos los grupos dado el patrón de texto creado
matches = re.finditer(pattern, chat, re.MULTILINE | re.VERBOSE)

## Creamos un diccionario en cada registro para hacerlo DataFrame 
df = pd.DataFrame([x.groupdict() for x in matches])

## Obtenemos la fecha de cada registro
df['Fecha'] = [''.join(re.findall('(\d{1,2}\/\d{1,2}\/\d{2},\s\d{2}:\d{2})',x)
                      ) for x in df['Fecha']]
## Y cambiamos al tipo correcto
df['Fecha'] = pd.to_datetime(df['Fecha'], format = '%m/%d/%y, %H:%M')

## Omitimos el \n de cada mensaje
df['Mensaje'] = df['Mensaje'].str.replace('\n', ' ')
df.sample(4)

,Fecha,Autor,Mensaje
13993,2020-06-02 10:06:00,Iván Jardón,No we
25582,2020-07-28 11:54:00,Iván Jardón,Me hizo bussines cases ahí mismo
30596,2020-08-09 20:35:00,Kevin Bacon,Chingonsisimo
14069,2020-06-02 21:06:00,Kevin Bacon,Que será 🤔


### Funciones

In [6]:
## Función para crear detalles de la fecha
def split_fecha(df,fecha):
    df[f'{fecha}_anio'] = pd.DatetimeIndex(df[fecha]).year
    df[f'{fecha}_mes'] = pd.DatetimeIndex(df[fecha]).month
    df[f'{fecha}_sem'] = df[fecha].dt.isocalendar().week
    df[f'{fecha}_diasem'] = df[fecha].dt.dayofweek + 1
    df[f'{fecha}_dia'] = df[fecha].dt.day
    df[f'{fecha}_hora'] = df[fecha].dt.hour
    df[f'{fecha}_minuto'] = df[fecha].dt.minute
    df['hora_min'] = df[f'{fecha}_hora'] + df[f'{fecha}_minuto']/60
    df[fecha] = df[fecha].dt.date

In [7]:
## Función para agrupar por día, con perc 25, 50 y 75 de la hora+minuto,
# y unir los mensajes además de contarlos
def group_fecha(df,grupo,hora_min,mensaje):
    df = df.groupby(grupo).agg({hora_min:[lambda x: np.percentile(x,10),
                                          lambda x: np.percentile(x,25),
                                          lambda x: np.percentile(x,50),
                                          lambda x: np.percentile(x,75),
                                          lambda x: np.percentile(x,90)],
                                mensaje:[sum,
                                         'count']})
    df.columns = [x + '_' + y for x, y in df.columns]
    df.rename(columns = {'hora_min_<lambda_0>':'hr_min_10',
                         'hora_min_<lambda_1>':'hr_min_25',
                         'hora_min_<lambda_2>':'hr_min_50',
                         'hora_min_<lambda_3>':'hr_min_75', 
                         'hora_min_<lambda_4>':'hr_min_90',
                         'Mensaje_sum':'Mensaje'}, 
              inplace = True)
    df.reset_index(inplace = True)
    return df

In [8]:
## Función para limpiar columnas de texto (también quita stopwords)
def clean_mensaje(df,mensaje):
        import nltk
        nltk.download('stopwords')
        from nltk.corpus import stopwords
        stop_words = stopwords.words('spanish')

        def clean_text(text, pattern='[^a-zA-Z]'):
            import re
            import unicodedata
            cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
            cleaned_text = re.sub(pattern, ' ', cleaned_text.decode('utf-8'), flags=re.UNICODE)
            cleaned_text = u' '.join(cleaned_text.lower().split())
            return cleaned_text
        
        texto = []
        for x in [n.split() for n in [clean_text(x) for x in df[mensaje]]]:
            aux = []
            for word in x:
                if word != 'a':
                    word = re.sub('^(j*a*)+(?=\s)','jaja',word + ' ') # arregla cualquier versión de jajaj
                    word = word.strip()
                    if word not in stop_words + ['media','omitted','https','www','com']:
                        aux.append(word)
            texto.append(aux)
        df[f'{mensaje}_limpio'] = [' '.join(x) for x in texto]

In [9]:
## Función para desplegar características del mensaje
def caract_msj(df,mensaje):
    df[f'{mensaje}_long'] = df[mensaje].str.len()
    df[f'{mensaje}_n_words'] = df[mensaje].str.split().str.len()
    df[f'{mensaje}_n_letters'] = df[mensaje].map(lambda x:sum(map(str.isalpha, x)))
    df[f'{mensaje}_n_whitespaces'] = df[mensaje].map(lambda x:len(re.findall('\s', x)))
    df[f'{mensaje}_n_media'] = df[mensaje].map(lambda x:len(re.findall('<Media omitted>', x)))
    df[f'{mensaje}_url'] = (df[mensaje].str.contains('http'))*1

    import emoji
    df[f'{mensaje}_n_emojis'] = df[mensaje].map(emoji.emoji_count)

In [10]:
## Se aplican todas las funciones
split_fecha(df,'Fecha')
grupo = ['Fecha','Autor','Fecha_anio','Fecha_mes','Fecha_sem','Fecha_diasem','Fecha_dia']
df = group_fecha(df,grupo,'hora_min','Mensaje')
clean_mensaje(df,'Mensaje')
caract_msj(df,'Mensaje')

[nltk_data] Downloading package stopwords to /home/ef/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
## Se guardan los tipos de variables
autores = sorted(np.unique(df['Autor']))
autores = dict(zip(autores,range(1,len(autores)+1)))
cat = [x for x in df.columns if x.startswith('Fecha_')]
num = [x for x in df.describe().columns if x not in cat]

## Modelaje

### Outliers

In [13]:
display(df[num].describe([.01,.1,.9,.99]))
dim_antes = len(df)

import whatsapp as wa # módulo personalizado, en validación se detalla
for col in num:
    df = wa.outlier(df,col)
    
display('{:.2%}'.format(len(df) / dim_antes))  
df[num].describe([.01,.1,.9,.99])

,hr_min_10,hr_min_25,hr_min_50,hr_min_75,hr_min_90,Mensaje_count,Mensaje_long,Mensaje_n_words,Mensaje_n_letters,Mensaje_n_whitespaces,Mensaje_n_media,Mensaje_url,Mensaje_n_emojis
count,1027.00,1027.00,1027.00,1027.00,1027.00,1027.00,1027.00,1027.00,1027.00,1027.00,1027.00,1027.00,1027.00
mean,11.21,12.80,15.03,17.13,18.66,51.19,1154.35,205.62,903.71,205.78,6.04,0.22,3.51
std,3.02,3.31,3.54,3.49,3.29,48.58,1139.40,206.63,895.12,206.84,5.17,0.41,5.33
min,0.03,0.25,0.38,0.73,8.37,1.00,7.00,1.00,6.00,1.00,0.00,0.00,0.00
1%,0.34,0.84,8.08,8.93,9.34,1.00,16.00,2.00,12.00,2.00,0.00,0.00,0.00
10%,8.52,9.46,10.59,12.33,14.13,6.60,123.80,19.60,94.00,19.60,1.00,0.00,0.00
50%,10.81,12.32,15.02,17.41,19.28,37.00,823.00,142.00,645.00,142.00,5.00,0.00,2.00
90%,15.31,17.41,19.65,21.63,22.25,117.00,2607.20,469.80,2044.80,472.60,13.00,1.00,10.00
99%,19.72,20.78,22.30,23.15,23.65,220.92,5190.32,934.44,4074.86,935.18,24.74,1.00,24.00
max,22.83,22.98,23.17,23.95,23.97,402.00,8503.00,1558.00,6696.00,1558.00,38.00,1.00,58.00


'98.25%'

,hr_min_10,hr_min_25,hr_min_50,hr_min_75,hr_min_90,Mensaje_count,Mensaje_long,Mensaje_n_words,Mensaje_n_letters,Mensaje_n_whitespaces,Mensaje_n_media,Mensaje_url,Mensaje_n_emojis
count,1009.00,1009.00,1009.00,1009.00,1009.00,1009.00,1009.00,1009.00,1009.00,1009.00,1009.00,1009.00,1009.00
mean,11.21,12.80,15.03,17.11,18.63,48.67,1096.93,195.31,858.82,195.46,5.82,0.21,3.11
std,3.03,3.32,3.56,3.52,3.31,43.61,1036.37,187.92,814.66,188.16,4.76,0.41,4.10
min,0.03,0.25,0.38,0.73,8.37,1.00,7.00,1.00,6.00,1.00,0.00,0.00,0.00
1%,0.33,0.68,8.07,8.86,9.33,1.00,16.00,2.00,12.00,2.00,0.00,0.00,0.00
10%,8.50,9.46,10.48,12.33,14.09,6.00,121.00,19.00,93.00,19.00,1.00,0.00,0.00
50%,10.79,12.32,15.02,17.38,19.25,37.00,800.00,139.00,624.00,141.00,5.00,0.00,2.00
90%,15.34,17.41,19.66,21.64,22.27,111.20,2501.00,452.20,1954.60,453.20,12.00,1.00,9.00
99%,19.80,20.78,22.30,23.16,23.66,188.84,4687.28,842.92,3740.64,842.92,21.00,1.00,18.00
max,22.83,22.98,23.17,23.95,23.97,275.00,5744.00,1054.00,4547.00,1054.00,26.00,1.00,22.00


### Objetivo

Quién es el autor?

In [14]:
## Se reemplazan los autores (ordenados alfabéticamente) por números
df['OBJETIVO'] = df['Autor'].replace(autores)

df[['Autor','OBJETIVO']].value_counts().reset_index().sort_values(by = 'OBJETIVO').iloc[:,:2]

,Autor,OBJETIVO
2,EF,1
0,Iván Jardón,2
1,Kevin Bacon,3


In [15]:
X = df[['Mensaje_limpio'] + cat + num]
y = df['OBJETIVO']

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size = 0.77, 
                                                    random_state = 22)

### Preprocesamiento

In [17]:
## Dummies para categóricas
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore')

## Escala para numéricas
from sklearn.preprocessing import MinMaxScaler
mm_x = MinMaxScaler()

## Frecuencia de palabras para texto
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(ngram_range = (1, 1), 
                     min_df = 10, 
                     max_features = 100)

## Se aplica transformación para cada tipo de columnas
from sklearn.compose import ColumnTransformer
prep = ColumnTransformer(transformers=[('OHE', ohe, cat),
                                       ('Scale', mm_x, num), 
                                       ('CountV', cv, 'Mensaje_limpio')])

In [18]:
## Top palabras por autor
wa.words(df,cv)

,EF,Iván Jardón,Kevin Bacon
0,jaja,jaja,jaja
1,si,si,si
2,ah,we,jardon
3,pa,wey,mas
4,abuebo,mas,brob
5,amigo,voy,ah
6,mas,bien,bien
7,brob,asi,we
8,we,kevin,bueno
9,bien,verga,bro


### Modelos

In [19]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

param_logreg = {'penalty':['l1', 'l2', 'elasticnet'], 
                'C':[x for x in range(1,11)], 
                'class_weight':['None','balanced'],
                'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
                }

from sklearn.model_selection import GridSearchCV
search_logreg = GridSearchCV(param_grid = param_logreg, 
                             cv = 4, 
                             n_jobs = -1, 
                             scoring = 'accuracy',
                             estimator = logreg,
                             verbose = 5)

In [20]:
from sklearn.ensemble import RandomForestClassifier 
forest = RandomForestClassifier()

param_forest = {'n_estimators': [x for x in range(1400, 1500, 30)],
                'max_features': ['auto', 'sqrt', 'log2'],
                'criterion': ['gini', 'entropy'],
                'class_weight': ['balanced', None],
                'min_samples_split': [x for x in range(17, 22)],
                'min_samples_leaf': [x/100 for x in range(1, 6)]
               }

from sklearn.model_selection import RandomizedSearchCV
search_forest = RandomizedSearchCV(param_distributions = param_forest, 
                                   cv = 4, 
                                   n_jobs = -1, 
                                   scoring = 'accuracy',
                                   estimator = forest,
                                   verbose = 5,
                                   n_iter = 10,
                                   random_state = 22)

In [21]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()

param_ada={'n_estimators':[x for x in range(50,250,50)],
           'learning_rate':[x/10 for x in range(1,11)]
          }

from sklearn.model_selection import RandomizedSearchCV
search_ada = RandomizedSearchCV(param_distributions = param_ada, 
                                cv = 4, 
                                n_jobs = -1, 
                                scoring = 'accuracy', 
                                estimator = ada, 
                                verbose = 5,
                                n_iter = 10,
                                random_state = 22)

In [30]:
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier()

param_xgb = {'learning_rate':[x/100 for x in range(1,111)],
             'n_estimators':[x for x in range(1,111)],
             'max_depth':[x for x in range(1,11)], 
             'min_child_weight':[x for x in range(1,111)],
             'objective':['count:poisson','multi:softmax'],
             'subsample':[x/100 for x in range(50,111)], 
             'colsample_bytree':[x/100 for x in range(50,111)], 
            }

from sklearn.model_selection import RandomizedSearchCV
search_xgb = RandomizedSearchCV(param_distributions = param_xgb, 
                                cv = 4, 
                                n_jobs = -1, 
                                scoring = 'accuracy', 
                                estimator = xgb, 
                                verbose = 5,
                                n_iter = 100,
                                random_state = 22)

### Voting

In [33]:
from sklearn.ensemble import VotingClassifier
vc = VotingClassifier(estimators = [('LogReg', search_logreg), 
                                    ('Forest', search_forest), 
                                    ('ADA', search_ada), 
                                    ('XGB',search_xgb)], 
                      voting = 'soft')

### Pipeline

In [34]:
from sklearn.pipeline import Pipeline
modelo = Pipeline(steps=[('preproc', prep),
                         ('modelo', vc)])

modelo.fit(X_train,y_train).score(X_test,y_test)

Fitting 4 folds for each of 300 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 575 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 1187 tasks      | elapsed:   46.0s
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:   46.4s finished
/home/ef/Desktop/diplomadus/diplomadus/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   35.8s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.1min finished


Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   23.7s finished


Fitting 4 folds for each of 100 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:   27.7s
[Parallel(n_jobs=-1)]: Done 294 tasks      | elapsed:   55.5s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  1.4min finished


0.8068669527896996

In [37]:
## Variables que más se usan para diferenciar al autor/a
wa.top_variables(vc,X_train)

Modelo,LogReg,Forest,ADA,XGB
0,Fecha_dia,hr_min_90,hr_min_90,hr_min_75
1,Fecha_mes,Mensaje_n_words,hr_min_75,Mensaje_n_words
2,Fecha_anio,Fecha_sem,hr_min_50,hr_min_90
3,hr_min_75,Fecha_dia,hr_min_25,Fecha_sem
4,hr_min_50,Mensaje_long,hr_min_10,Fecha_diasem
5,hr_min_25,Fecha_diasem,Mensaje_url,Mensaje_count
6,hr_min_10,hr_min_25,Mensaje_n_words,Fecha_dia
7,Mensaje_url,hr_min_10,Mensaje_n_whitespaces,hr_min_25
8,Mensaje_n_words,Mensaje_count,Mensaje_n_media,Mensaje_long
9,Mensaje_n_whitespaces,hr_min_75,Mensaje_n_letters,hr_min_50


In [38]:
## En el conjunto de train
from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(confusion_matrix(y_true = y_train,
                                   y_pred = modelo.predict(X_train))/len(y_train), 
                  index = [{y: x for x, y in autores.items()
                           }[n] for n in list(sorted(np.unique(y_train)))], 
                  columns = [{y: x for x, y in autores.items()
                             }[n] for n in list(sorted(np.unique(y_train)))])
display(cm)

## Con buena acertividad (suma de diagonal en la matriz de confusión)
'Accuracy de {:.2%}'.format(np.asarray(cm).trace())

,EF,Iván Jardón,Kevin Bacon
EF,0.33,0.00,0.01
Iván Jardón,0.00,0.33,0.00
Kevin Bacon,0.00,0.00,0.33


'Accuracy de 99.10%'

In [39]:
## Y en test
cm = pd.DataFrame(confusion_matrix(y_true = y_test,
                                   y_pred = modelo.predict(X_test))/len(y_test), 
                  index = [{y: x for x, y in autores.items()
                           }[n] for n in list(sorted(np.unique(y_test)))], 
                  columns = [{y: x for x, y in autores.items()
                             }[n] for n in list(sorted(np.unique(y_test)))])
display(cm)

## Con buena acertividad (suma de diagonal en la matriz de confusión)
'Accuracy de {:.2%}'.format(np.asarray(cm).trace())

,EF,Iván Jardón,Kevin Bacon
EF,0.28,0.02,0.03
Iván Jardón,0.03,0.27,0.04
Kevin Bacon,0.05,0.03,0.26


'Accuracy de 80.69%'

In [40]:
## Guardar OHE, MinMax y modelo
import pickle
with open('modelo_whatsapp.pkl', "wb") as f:
    pickle.dump(modelo, f)

## Validación

In [41]:
## Abrir el pickle con lo necesario para validar
import pickle    
with open('modelo_whatsapp.pkl', "rb") as f:
    modelo = pickle.load(f)

## Listo para usarse
display('Transformadores:')
display([x[1] for x in modelo.get_params()['steps'][0][1].get_params()['transformers']])
display('Modelos:')
[x.best_estimator_ for x in modelo.get_params()['modelo'].estimators_]

'Transformadores:'

[OneHotEncoder(handle_unknown='ignore', sparse=False),
 MinMaxScaler(),
 TfidfVectorizer(max_features=100, min_df=10)]

'Modelos:'

[LogisticRegression(C=1, class_weight='balanced', penalty='l1', solver='saga'),
 RandomForestClassifier(criterion='entropy', min_samples_leaf=0.01,
                        min_samples_split=18, n_estimators=1400),
 AdaBoostClassifier(learning_rate=0.1, n_estimators=100),
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=0.65, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.45, max_delta_step=0, max_depth=5,
               min_child_weight=2, missing=nan, monotone_constraints='()',
               n_estimators=66, n_jobs=0, num_parallel_tree=1,
               objective='multi:softprob', random_state=0, reg_alpha=0,
               reg_lambda=1, scale_pos_weight=None, subsample=0.8,
               tree_method='exact', validate_parameters=1, verbosity=None)]

In [42]:
# %load basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',200)
pd.options.display.float_format = '{:.2f}'.format
file = '/home/ef/Documents/Diplomado/data/WhatsApp Chat with Naps 🐻🐼🐯.txt'

## Podemos crear un módulo con funciones y clases que ejecuten todo el proceso anterior
import whatsapp as wa
df = wa.read_chat(file)

## Transformación y obtención de tipos de variables
df,cat,num,autores = wa.TAD().transform(df)

## Se estructura y=f(X)
df['OBJETIVO'] = df['Autor'].replace(autores)
X = df[['Mensaje_limpio'] + cat + num]
y = df['OBJETIVO']

## Se predice sobre datos nuevos
val = df.join(pd.DataFrame(modelo.predict(X),
                           columns = ['Estimado']
                          ).replace({y: x for x, y in autores.items()}))

## Qué acertividad hay en la validación?
from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(confusion_matrix(y_true = val['Autor'],
                                   y_pred = val['Estimado'])/len(val), 
                  index = [x for x in autores], 
                  columns = [x for x in autores])
display(cm)

## Con buena acertividad (suma de diagonal en la matriz de confusión)
'Accuracy de {:.2%}'.format(np.asarray(cm).trace())

[nltk_data] Downloading package stopwords to /home/ef/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,EF,Iván Jardón,Kevin Bacon
EF,0.32,0.00,0.01
Iván Jardón,0.02,0.30,0.01
Kevin Bacon,0.01,0.01,0.32


'Accuracy de 93.77%'

## Fin

In [43]:
import math
def time_exp(x):
    print(str(int(math.floor(x/60))
             ) + " minutos con " + '{:.2f}'.format(60*(x/60 - math.floor(x/60))
                                                  ) + " segundos")
end = time.time()
time_exp(end - start)

21 minutos con 44.55 segundos


In [44]:
## Tono para cuando termina código
from IPython.lib.display import Audio
import numpy as np

framerate = 4410
play_time_seconds = 1

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(5*np.pi*300*t) + np.sin(2*np.pi*240*t)

## La siguiente línea debe ir debajo del código p que suene
Audio(audio_data, rate=framerate, autoplay=True)